In [1]:
!python -m pip install --user --upgrade pip
!pip install --user keras
!pip install --user tensorflow
!pip install --user -U numpy
!pip install --user hyperas

Requirement already up-to-date: pip in c:\users\wangmi94\appdata\roaming\python\python36\site-packages (19.2.2)
Requirement already up-to-date: numpy in c:\users\wangmi94\appdata\roaming\python\python36\site-packages (1.17.0)


In [ ]:
#!/usr/bin/python
#-*- coding: UTF-8 -*-

In [ ]:
# Use CPU only
# Only Macbook needs to run this cell
#Uncomment following lines to run
#import os
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
#os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [3]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dense, LeakyReLU
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn import preprocessing
import matplotlib.pyplot as plt
#from keras.layers.normalization import BatchNormalization
from keras import backend as K
import sklearn.metrics as skm, math
from keras.layers.advanced_activations import PReLU, ELU
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

## Load data and do data normalization

In [4]:
# df = pd.read_csv('Options_R_new.csv')
# df = df.dropna()
# df.head()

In [5]:
def data():
    df = pd.read_csv('Options_R_new.csv')
    df = df.dropna()
    
    # Remove the option data has less than 7 days to maturity.
    df = df[df.maturity > 6]

    # Remove the option data with deep in-the-money and deep out-of-money
    def moneyness(S,X):
        return S/X
    
    df['moneyness'] = df.apply(lambda row: moneyness(row['underlying_price'], row['strike_price']), axis = 1)
    df = df[df.moneyness >= 0.7]
    df = df[df.moneyness <= 1.2]
    
    def intrinsic(S, K, C, P):
        return C*(max(S-K, 0)) + P*(max(K-S, 0))
    
    df['intrinsic'] = df.apply(lambda row: intrinsic(row['underlying_price'], row['strike_price'], row['cp_flag_C'], row['cp_flag_P']), axis = 1)
    
    sample_df = df.sample(n = 50000)
    
    y = sample_df['best_offer'].values
    X = sample_df[['maturity', 'strike_price', 'impl_volatility', 'underlying_price', 'cp_flag_C', 'interest_rate','intrinsic']]
    
    # Normalize the data to build a regression neural network model
    X = preprocessing.normalize(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    #X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    
    return X_train, y_train, X_test, y_test

## Build a keras Sequential model

In [6]:
# Because it is a simple regression problem, we should custom metrics function
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis = -1))

def r_square(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return (1 - SS_res/(SS_tot + K.epsilon()))

def mpe(y_true, y_pred):
    errors = K.sqrt(K.mean((y_true-y_pred)**2))/K.mean(y_true)
    return errors

def exp(x):
    return K.exp(x)

# Enable early stopping based on the loss of validation data
es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=50)

In [7]:
def model(X_train, y_train, X_test, y_test):
    
#     def rmse(y_true, y_pred):
#         return K.sqrt(K.mean(K.square(y_pred - y_true), axis = -1))

    def r_square(y_true, y_pred):
        SS_res =  K.sum(K.square(y_true - y_pred)) 
        SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
        return (1 - SS_res/(SS_tot + K.epsilon()))

#     def mpe(y_true, y_pred):
#         errors = K.sqrt(K.mean((y_true-y_pred)**2))/K.mean(y_true)
#         return errors

    def exp(x):
        return K.exp(x)
    
    model = Sequential()
    model.add(Dense(32, input_shape = (7,)))
    model.add(LeakyReLU(alpha = 0.1))
    
    model.add(Dense({{choice([32, 64, 128, 256])}}))
    model.add(Activation('relu'))
    
    model.add(Dense({{choice([64, 128, 256])}}))
    model.add(Activation({{choice([ELU(), 'relu'])}}))
        
    model.add(Dense(1, activation = exp))
    
    es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=25)
        
    model.compile(loss = 'mse', metrics = [r_square], optimizer = {{choice(['rmsprop', 'adam'])}})
    
    result = model.fit(X_train, y_train,
                       batch_size = {{choice([64, 128, 256])}},
                       nb_epoch = 200,
                       verbose = 2,
                       validation_split = 0.1,
                       callbacks = [es])
    validation_acc = np.amax(result.history['val_r_square'])
    
    print('Best validation accuracy of epoch:', validation_acc)
    
    return {'loss': validation_acc, 'status': STATUS_OK, 'model': model}

In [8]:
a = 'OPPT'
print(type(a))

# str------>bytes(gbk)
b_utf8 = a.encode("charmap")
print(b_utf8)

d_unicode = b_utf8.decode("charmap")
print(d_unicode)

<class 'str'>
b'OPPT'
OPPT


In [9]:
best_run, best_model = optim.minimize(model = model,
                                      data = data,
                                      algo=tpe.suggest,
                                      max_evals = 30,
                                      trials = Trials(),
                                      notebook_name = 'OP_Parameter_Tuning_with_cp')
X_train, y_train, X_test, y_test = data()

>>> Imports:
#coding=utf-8

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers import Activation, Dense, LeakyReLU
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from keras.callbacks import EarlyStopping
except:
    pass

try:
    from sklearn import preprocessing
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    import sklearn.metrics, math as skm
except:
    pass

try:
    from keras.layers.advanced_activations import PReLU, ELU
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    import talos
except:
    pass

try:
    from talos.

W0821 21:40:25.488411 14968 deprecation_wrapper.py:119] From C:\Users\wangmi94\AppData\Roaming\Python\Python36\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



  0%|                                                                             | 0/30 [00:00<?, ?it/s, best loss: ?]

W0821 21:40:25.551538 14968 deprecation_wrapper.py:119] From C:\Users\wangmi94\AppData\Roaming\Python\Python36\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0821 21:40:25.566248 14968 deprecation_wrapper.py:119] From C:\Users\wangmi94\AppData\Roaming\Python\Python36\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0821 21:40:25.622254 14968 deprecation_wrapper.py:119] From C:\Users\wangmi94\AppData\Roaming\Python\Python36\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W:\research\Option-Pricing\Neural Network Model\temp_model.py:160: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.

W0821 21:40:25.774791 14968 deprecation_wrapper.py:119] From C:\Users\wangmi94\AppData\Roaming\Python\Python36\site-p

Train on 36000 samples, validate on 4000 samples                                                                       
Epoch 1/200                                                                                                            
 - 1s - loss: 60.0855 - r_square: 0.3762 - val_loss: 30.6226 - val_r_square: 0.6438                                    

Epoch 2/200                                                                                                            
 - 0s - loss: 29.4526 - r_square: 0.6772 - val_loss: 58.6772 - val_r_square: 0.3253                                    

Epoch 3/200                                                                                                            
 - 0s - loss: 26.5421 - r_square: 0.7139 - val_loss: 24.8566 - val_r_square: 0.7196                                    

Epoch 4/200                                                                                                            
 - 0s - loss: 23.9139 - r_square: 0.7

...

 - 0s - loss: 29.6890 - r_square: 0.6912 - val_loss: 23.4897 - val_r_square: 0.7447                                    

Epoch 5/200                                                                                                            
 - 1s - loss: 27.6133 - r_square: 0.7144 - val_loss: 25.3022 - val_r_square: 0.7265                                    

Epoch 6/200                                                                                                            
 - 0s - loss: 27.0443 - r_square: 0.7161 - val_loss: 23.9509 - val_r_square: 0.7411                                    

Epoch 7/200                                                                                                            
 - 0s - loss: 26.4587 - r_square: 0.7256 - val_loss: 26.0277 - val_r_square: 0.7142                                    

Epoch 8/200                                                                                                            
 - 1s - loss: 25.6297 - r_square: 0.

Epoch 38/200                                                                                                           
 - 0s - loss: 27.5856 - r_square: 0.7267 - val_loss: 24.0340 - val_r_square: 0.7436                                    

Epoch 39/200                                                                                                           
 - 0s - loss: 32.0070 - r_square: 0.6755 - val_loss: 42.4247 - val_r_square: 0.5459                                    

Epoch 40/200                                                                                                           
 - 0s - loss: 33.2542 - r_square: 0.6655 - val_loss: 15.3481 - val_r_square: 0.8341                                    

Epoch 41/200                                                                                                           
 - 0s - loss: 39.8628 - r_square: 0.5846 - val_loss: 77.8448 - val_r_square: 0.1656                                    

Epoch 42/200                        

Epoch 5/200                                                                                                            
 - 0s - loss: 29.8807 - r_square: 0.6924 - val_loss: 22.7765 - val_r_square: 0.7523                                    

Epoch 6/200                                                                                                            
 - 0s - loss: 27.5249 - r_square: 0.7130 - val_loss: 21.5332 - val_r_square: 0.7655                                    

Epoch 7/200                                                                                                            
 - 0s - loss: 26.4124 - r_square: 0.7277 - val_loss: 29.1998 - val_r_square: 0.6866                                    

Epoch 8/200                                                                                                            
 - 0s - loss: 25.1059 - r_square: 0.7373 - val_loss: 26.7048 - val_r_square: 0.7128                                    

Epoch 9/200                         

 - 0s - loss: 16.0307 - r_square: 0.8330 - val_loss: 14.5000 - val_r_square: 0.8428                                    

Epoch 39/200                                                                                                           
 - 0s - loss: 15.6023 - r_square: 0.8374 - val_loss: 18.5276 - val_r_square: 0.7978                                    

Epoch 40/200                                                                                                           
 - 0s - loss: 15.9559 - r_square: 0.8330 - val_loss: 25.2662 - val_r_square: 0.7291                                    

Epoch 41/200                                                                                                           
 - 0s - loss: 15.5147 - r_square: 0.8397 - val_loss: 25.6939 - val_r_square: 0.7234                                    

Epoch 42/200                                                                                                           
 - 0s - loss: 15.3294 - r_square: 0.

Epoch 72/200                                                                                                           
 - 0s - loss: 13.1717 - r_square: 0.8621 - val_loss: 11.3171 - val_r_square: 0.8784                                    

Epoch 73/200                                                                                                           
 - 0s - loss: 13.4004 - r_square: 0.8598 - val_loss: 15.3475 - val_r_square: 0.8366                                    

Epoch 74/200                                                                                                           
 - 0s - loss: 13.2370 - r_square: 0.8618 - val_loss: 21.9835 - val_r_square: 0.7646                                    

Epoch 75/200                                                                                                           
 - 0s - loss: 13.5610 - r_square: 0.8585 - val_loss: 10.9264 - val_r_square: 0.8816                                    

Epoch 76/200                        

In [10]:
print("Evalutation of best performing model:")
print(best_model.evaluate(X_test, y_test))
print("Best performing model chosen hyper-parameters:")
print(best_run)

Evalutation of best performing model:
10000/10000 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 36us/step
[94.55253149719238, 0.03706799507141113]
Best performing model chosen hyper-parameters:
{'Activation': 1, 'Dense': 3, 'Dense_1': 2, 'Dense_2': 2, 'optimizer': 0}


In [11]:
y_pred = best_model.predict(X_test)
print(y_pred[:10])
print(y_test[:10])

[[2.3177297 ]
 [0.5825144 ]
 [5.2768426 ]
 [5.876446  ]
 [0.22862577]
 [3.3820906 ]
 [0.268649  ]
 [1.0001414 ]
 [1.8926716 ]
 [2.5196552 ]]
[10.    3.35  9.1  13.35  0.27  6.4   0.06  1.98  1.64  2.12]


In [12]:
best_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_113 (Dense)            (None, 32)                256       
_________________________________________________________________
leaky_re_lu_29 (LeakyReLU)   (None, 32)                0         
_________________________________________________________________
dense_114 (Dense)            (None, 256)               8448      
_________________________________________________________________
activation_57 (Activation)   (None, 256)               0         
_________________________________________________________________
dense_115 (Dense)            (None, 256)               65792     
_________________________________________________________________
activation_58 (Activation)   (None, 256)               0         
_________________________________________________________________
dense_116 (Dense)            (None, 1)                 257       
Total para

In [13]:
def mean_percent_error(y_true, y_pred):
    errors = np.sqrt(np.mean((y_true-y_pred)**2))/np.mean(y_true)
    return errors

In [14]:
print("Average precentage pricing error: %f" % mean_percent_error(y_test, y_pred))
print("Mean absolute error (MAE):      %f" % skm.mean_absolute_error(y_test, y_pred))
print("Mean squared error (MSE):       %f" % skm.mean_squared_error(y_test, y_pred))
print("Root mean squared error (RMSE): %f" % math.sqrt(skm.mean_squared_error(y_test, y_pred)))
print("R square (R^2):                 %f" % skm.r2_score(y_test, y_pred))

Average precentage pricing error: 1.830763
Mean absolute error (MAE):      4.620960
Mean squared error (MSE):       94.552532
Root mean squared error (RMSE): 9.723813
R square (R^2):                 0.052304


In [15]:
#To satisfy the req of talos, scipy must be in version 1.2 and above
#Run to check your version/ download
!pip install --user scipy

In [16]:
!pip install --user talos

  Created wheel for talos: filename=talos-0.5.0-cp36-none-any.whl size=41550 sha256=5d31ecd641a7f637d115b7c1af909c44ff69504ecb0fc7dfee5580c95fad707d
  Stored in directory: C:\Users\wangmi94\AppData\Local\pip\Cache\wheels\fc\19\4a\5e72a9d4c2015760295561268d22f60c01c7cc599faf560ea3
  Created wheel for astetik: filename=astetik-1.9.8-cp36-none-any.whl size=56073 sha256=32ac1c0384aafed4b62969f2916ac4e4a910a2956c22f40372a1ca259c8bd084
  Stored in directory: C:\Users\wangmi94\AppData\Local\pip\Cache\wheels\ba\cc\e9\11c6a853d8379f295e17b68f2139ea1bbcd13c5b260822abc7
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1324 sha256=75ced3d6b811f3ab9c1a4fc73cc399f5d61924c70592ccf0f5cacfaeb3334742
  Stored in directory: C:\Users\wangmi94\AppData\Local\pip\Cache\wheels\76\03\bb\589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
  Created wheel for chances: filename=chances-0.1.6-cp36-none-any.whl size=52555 sha256=b19730589ac82bb4fe7a9b2979d7eb83196a36e97bc18146c93a5e47208a0

ERROR: wrangle 0.6.7 has requirement scipy==1.2, but you'll have scipy 1.1.0 which is incompatible.


In [17]:
import talos

C:\Program Files\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [18]:
df = pd.read_csv('Options_R_new.csv')
df = df.dropna()
    
# Remove the option data has less than 7 days to maturity.
df = df[df.maturity > 6]

# Remove the option data with deep in-the-money and deep out-of-money
def moneyness(S,X):
    return S/X
    
df['moneyness'] = df.apply(lambda row: moneyness(row['underlying_price'], row['strike_price']), axis = 1)
df = df[df.moneyness >= 0.7]
df = df[df.moneyness <= 1.2]
    
def intrinsic(S, K, C, P):
    return C*(max(S-K, 0)) + P*(max(K-S, 0))
    
df['intrinsic'] = df.apply(lambda row: intrinsic(row['underlying_price'], row['strike_price'], row['cp_flag_C'], row['cp_flag_P']), axis = 1)
    
sample_df = df.sample(n = 500000)
    
y = sample_df['best_offer'].values
X = sample_df[['maturity', 'strike_price', 'impl_volatility', 'underlying_price', 'cp_flag_C', 'cp_flag_P', 'interest_rate']]
    
# Normalize the data to build a regression neural network model
X = preprocessing.normalize(X)

In [19]:
X.shape

(500000, 7)

In [20]:
def talos_model(X_train, y_train, X_val, y_val, params):
    model = Sequential()
    model.add(Dense(32, input_shape=(7,), activation=params['activation'], kernel_initializer='normal'))

    model.add(Dense(params['second_neuron'], activation = params['activation'], kernel_initializer = 'normal'))
    
    model.add(Dense(params['third_neuron'], activation = params['activation'], kernel_initializer = 'normal'))
    
    model.add(Dense(1, activation=params['last_activation'], kernel_initializer='normal'))
    
    model.compile(loss=params['losses'],
                  # here we add a regulizer normalization function from Talos
                  optimizer=params['optimizer'], #(lr=lr_normalizer(params['lr'],params['optimizer'])),
                  metrics=[r_square, mpe])
    
    out = model.fit(X_train, y_train,
                    validation_data = [X_val, y_val],
                    batch_size = params['batch_size'],
                    nb_epoch = params['epochs'],
                    verbose = 2)
    return out, model

In [21]:
p = {#'lr': (0.5, 5, 10),
     'second_neuron':[32, 64, 128, 256],
     'third_neuron':[64, 128, 256],
     'batch_size': [128, 256],
     'epochs': [150],
     #'dropout': (0, 0.5, 5),
     #'weight_regulizer':[None],
     #'emb_output_dims': [None],
     #'shape':['brick','long_funnel'],
     'optimizer': ['Adam', 'Nadam', 'RMSprop'],
     'losses': ['mse'],
     'activation':['relu', 'elu'],
     'last_activation': ['linear', exp]}

In [23]:
t = talos.Scan(x=X, y=y, params = p, model=talos_model, grid_downsample=0.1)

  0%|                                                                                           | 0/28 [00:00<?, ?it/s]

Train on 350000 samples, validate on 150000 samples
Epoch 1/150
 - 4s - loss: 75.8773 - r_square: 0.1945 - mpe: 1.4408 - val_loss: 37.8964 - val_r_square: 0.5776 - val_mpe: 1.0421
Epoch 2/150
 - 4s - loss: 29.7715 - r_square: 0.6671 - mpe: 0.8997 - val_loss: 13.7442 - val_r_square: 0.8435 - val_mpe: 0.6294
Epoch 3/150
 - 4s - loss: 17.3708 - r_square: 0.8064 - mpe: 0.6726 - val_loss: 11.0989 - val_r_square: 0.8743 - val_mpe: 0.5655
Epoch 4/150


KeyboardInterrupt: 

In [24]:
# accessing the results data frame
t.data.head(28)

,round_epochs,val_loss,val_r_square,val_mpe,loss,r_square,mpe,second_neuron,third_neuron,batch_size,epochs,optimizer,losses,activation,last_activation
0,150,0.129232,9.169395e-01,0.061013,0.202083,4.489369e-01,0.073855,256,256,128,150,Adam,mse,relu,linear
1,150,0.209024,5.161473e-01,0.077802,0.315827,2.697674e-01,0.092047,32,64,128,150,Adam,mse,elu,<function exp at 0x000001E4291BA730>
2,150,0.143381,-6.127050e-03,0.064272,0.201563,-5.025088e-03,0.073211,256,64,256,150,Nadam,mse,elu,linear
3,150,0.132300,4.464433e-01,0.061685,0.210404,1.571601e-01,0.075413,128,128,256,150,Nadam,mse,elu,linear
4,150,0.293439,2.613736e-01,0.091796,0.534121,1.996242e-01,0.118060,64,128,128,150,RMSprop,mse,relu,linear
5,150,0.236364,5.402571e-01,0.082121,0.544530,2.247692e-01,0.117134,128,128,128,150,RMSprop,mse,relu,<function exp at 0x000001E4291BA730>
6,150,0.303661,6.150180e-02,0.093627,0.394304,2.257027e-01,0.103742,256,64,256,150,Nadam,mse,relu,linear
7,150,0.487340,-9.571702e+14,0.118114,1.282802,-9.015574e+14,0.178111,64,256,128,150,RMSprop,mse,relu,linear
8,150,0.228135,4.451217e-01,0.080988,0.299191,1.764832e-01,0.090880,32,256,256,150,Adam,mse,relu,<function exp at 0x000001E4291BA730>
9,150,0.576733,1.085392e-01,0.128790,1.607607,6.323438e-02,0.206025,32,128,256,150,RMSprop,mse,elu,linear


In [25]:
t.peak_epochs_df

,val_loss,val_r_square,val_mpe,loss,r_square,mpe,acc_epoch,loss_epoch
1,120,0,120,144,0,144,NaN,0.325968
2,137,0,137,147,0,147,NaN,0.026704
3,121,8,121,143,10,143,NaN,0.037512
4,144,3,144,134,0,134,NaN,0.667039
5,141,2,141,149,0,149,NaN,0.350180
6,143,0,140,149,0,149,NaN,0.272806
7,124,0,124,146,0,146,NaN,0.346270
8,50,149,50,42,149,44,NaN,0.001549
9,141,0,141,144,0,144,NaN,0.137718
10,116,0,116,104,0,102,NaN,0.056146


In [26]:
from talos.utils.best_model import best_model
from talos.utils.best_model import activate_model
model_id = best_model(t, 'val_loss', True)
model = activate_model(t, model_id)

In [27]:
y_pred = model.predict(X)

In [28]:
y_pred[:10]

array([[ 0.26371574],
       [ 0.07901074],
       [13.984449  ],
       [12.6594925 ],
       [24.91445   ],
       [ 0.9355886 ],
       [ 0.71376014],
       [ 1.2284801 ],
       [-0.06573556],
       [ 0.33717322]], dtype=float32)

In [29]:
y[:10]

array([ 0.16,  0.23, 14.8 , 13.4 , 25.6 ,  0.72,  0.5 ,  0.83,  0.11,
        0.37])

In [30]:
#print("Average precentage pricing error: %f" % mean_percent_error(y, y_pred))
print("Mean absolute error (MAE):      %f" % skm.mean_absolute_error(y, y_pred))
print("Mean squared error (MSE):       %f" % skm.mean_squared_error(y, y_pred))
print("Root mean squared error (RMSE): %f" % math.sqrt(skm.mean_squared_error(y, y_pred)))
print("R square (R^2):                 %f" % skm.r2_score(y, y_pred))

Mean absolute error (MAE):      0.200216
Mean squared error (MSE):       0.119293
Root mean squared error (RMSE): 0.345389
R square (R^2):                 0.998762


In [31]:
# access the summary details
t.details

experiment_name           082119222633_
random_method          uniform_mersenne
reduction_method                   None
reduction_interval                   50
reduction_window                     20
grid_downsample                     0.1
reduction_threshold                 0.2
reduction_metric                val_acc
reduce_loss                       False
complete_time            08/22/19/03:11
x_shape                     (500000, 7)
y_shape                       (500000,)
dtype: object

In [32]:
talos.Deploy(t, 'option_price_1', 'r_square');

Deploy package option_price_1 have been saved.


In [34]:
model.save('tune_cp.h5')

## Reference: 
&emsp;&emsp;What does 'Accuracy' mean in Regression? https://github.com/keras-team/keras/issues/7947 <br>
&emsp;&emsp;https://keras.io/metrics/<br>
&emsp;&emsp;Scale, Standardize, or Normalize with Scikit-Learn https://towardsdatascience.com/scale-standardize-or-normalize-with-scikit-learn-6ccc7d176a02 <br>
&emsp;&emsp;The Day my Computer Won the Nobel Prize (Neural Network Option Pricing)  https://medium.com/datadriveninvestor/the-day-my-computer-won-the-nobel-prize-neural-network-option-pricing-d29b4379f1d2 <br>